In [1]:
import os
import numpy as np
import imageio
from matplotlib import pyplot as plt
from utils import *

In [2]:
tMax = 60
stockMin = 317
stockMax = 363
strike = 340

###
n = int((stockMax - stockMin)/0.01 + 1)
stock = np.linspace(stockMin, stockMax, n)

### t days until expiration
valueT = np.zeros((tMax+1, n))
for t in range(tMax+1):
    value = BlackScholes(strike, stock, t)
    valueT[t] = value

### Normalizing Data

# normalizing stock value
x = (stock - strike) / strike

# normalizing contract value
idx = np.argmin(np.abs(stock - strike))
yArr = valueT / valueT[tMax, idx]
yMax = np.max(yArr)
y0 = yArr[0]

In [3]:
# Generating images
files = []
for t in range(tMax+1)[::-1]:
    y = yArr[t]
    plt.figure(figsize = [8, 8], tight_layout=True)
    plt.plot(x, y, color = 'r', linewidth = 2.5, label = "Total Contract Value")
    plt.plot(x, y0, color = 'k', linewidth = 2.5, linestyle = 'dashed', label = "Intrinsic Contract Value")
    xlabel = r"$\bf{Value\ of\ the\ Underyling\ Asset}$"
    xlabel += "\n"
    xlabel += r"$\rm{(Percent\ difference\ between\ stock\ price\ and\ strike\ price)}$"
    plt.xlabel(xlabel, fontsize = 16)
    ylabel = r"$\bf{Value\ of\ the\ Option\ Contract}$"
    ylabel += "\n"
    ylabel += r"$\rm{(Ratio\ of\ contract\ price\ to\ %dDTE\ ATM\ contract\ price)}$" % tMax
    plt.ylabel(ylabel, fontsize = 16)
    title = r"$\bf{Theta\ Decay\ of\ Call\ Option\ Contracts}$"
    title += "\n"
    title += r"$\rm{Days\ to\ Expiration:}$"
    title += " %2s" % t
    plt.title(title, family = "monospace", fontsize = 16)
    plt.legend(fontsize = 16, loc=2)
    plt.ylim(-0.05, yMax)
    plt.xlim(x[0], x[-1])
    plt.xticks(fontsize = 14)
    plt.yticks(fontsize = 14)
    plt.grid()
    plt.gca().set_xticklabels(['{:.0f}%'.format(x*100) for x in plt.gca().get_xticks()]);
    
    fname = "option_%02d.png" % t
    files.append(fname)
    plt.savefig(fname)
    plt.close()

In [4]:
### Making movie animation
images = []
for i in range(tMax+1):
    fname = files[i]
    # make first and last frames longer
    if i == 0 or i == tMax:
        for _ in range(9):
            images.append(imageio.imread(fname))
    #
    images.append(imageio.imread(fname))
imageio.mimsave('OptionDecay.gif', images, duration=0.1)

### cleanup
for fname in files:
    os.remove(fname)